<img src="https://i1.sndcdn.com/avatars-000274765548-vj7h0w-t500x500.jpg" style='float:right; width:200px; margin: 0 20px;'>

<img src="https://cdn2.iconfinder.com/data/icons/popular-social-media-flat/48/Popular_Social_Media-11-512.png" style='float:right; width:200px; margin: 0 20px;'>


# Twitter V CMV topological differences --> Twitter
---

> Create feature set for CMV and Twitter 

> Do comperative analysis of features

In [ ]:
# env 
import sys
sys.path.append('/Users/shaimeital/code/thesis/conversant')
sys.path.append('/Users/shaimeital/code/thesis/cmv-stance-classification')


In [1]:
from typing import Iterable, List, Sequence, Set
import json

import pandas as pd
import numpy as np
import networkx as nx
import glob
import os
import csv

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [2]:
from conversant.conversation.examples.controversial_feature_extraction import *
from conversant.conversation\
    .conversation_utils import conversation_to_dataframe
from stance_classification.twitter_conversation_parser import TwitterConversationReader

from stance_classification.classifiers.maxcut_stance_classifier import MaxcutStanceClassifier

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Load Dataset
First load the gold standard,
then load the conversations for which we have gold-standard annotations

## Gold Standard

In [8]:
def load_annotations(annotations_path: str) -> Iterable[dict]:
    with open(annotations_path, 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for record in reader:
            yield {
                "cid": record[0],
                "author": record[1],
                "label": record[2]
            }

In [9]:
path = r"C:\Users\ronp\Documents\stance-classification\stance-gs-extended.csv"

annotations = load_annotations(path)
labeled_df = pd.DataFrame.from_records(annotations)
labeled_df

,cid,author,label
0,4rl42j,ThatBelligerentSloth,1
1,4rl42j,Hq3473,-1
2,4rl42j,Mintnose,-1
3,4rl42j,Octobers_second_one,-1
4,4rl42j,yaxamie,1
...,...,...,...
147,6l6xwb,forerunner398,-1
148,6lksmn,FrostbiteOrange,1
149,6lksmn,VertigoOne,-1
150,6lksmn,muyamable,-1


take conversations for which we have at least a single annotation

In [10]:
annotated_conversation_ids = sorted(set(labeled_df["cid"]))
print(f"{len(annotated_conversation_ids)} conversations with annotations")

13 conversations with annotations


## Load Raw Data
only relevant conversations that occurs in the gold standard

In [8]:
cmv_conversations_path = r"C:\Users\ronp\Documents\stance-classification\trees2.0.txt"
annotated_trees =
with open(cmv_conversations_path, 'r') as f:
    for line in f:
        tree = json.loads(f)
        if tree["id"] in annotated_conversation_ids:

available_conversation_ids = os.listdir(twitter_conversations_path)
twitter_conversation_ids = sorted(set(annotated_conversation_ids).intersection(available_conversation_ids))
print(f"{len(twitter_conversation_ids)} scraped conversations with annotations")

297 scraped conversations with annotations


In [9]:
# one example
conv_id = "498235547685756928"
conversation_path = os.path.join(twitter_conversations_path, conv_id, "tweets.jsonl")
sample = pd.read_json(conversation_path,
                     lines= True,
                     dtype={'in_reply_to_status_id_str':str, 'conversation_id_str': str, 'full_text':str, 'id_str':str})
sample

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


,created_at,id_str,full_text,display_text_range,entities,extended_entities,source,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,user_id_str,retweet_count,favorite_count,reply_count,quote_count,conversation_id_str,possibly_sensitive_editable,lang
0,2014-08-10 00:32:13+00:00,498265524397432832,@annaxsweat http://t.co/Z7Qm4iaoSR,"[0, 34]",{'user_mentions': [{'screen_name': 'annaxsweat...,"{'media': [{'id_str': '498265523889930242', 'i...","<a href=""http://twitter.com/#!/download/ipad"" ...",498235547685756928,1.539100e+09,annaxsweat,21461995,2,1,0,0,498235547685756928,1.0,und
1,2014-08-10 00:37:23+00:00,498266827676741632,@annaxsweat #StopThugCops We have to take acti...,"[0, 81]","{'hashtags': [{'text': 'StopThugCops', 'indice...",NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",498235547685756928,1.539100e+09,annaxsweat,21461995,4,2,0,0,498235547685756928,NaN,en
2,2014-08-11 05:58:51+00:00,498710112358244352,@annaxsweat 18*,"[0, 15]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,481509183,0,0,1,0,498235547685756928,NaN,und
3,2014-08-10 17:21:03+00:00,498519407375941632,@annaxsweat: Soooo wonderfully said! Thank you...,"[0, 73]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,2331498373,0,0,0,0,498235547685756928,NaN,en
4,2014-08-10 03:40:28+00:00,498312902928244736,"@annaxsweat well, he damn sure ain't a child","[0, 44]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,2437571275,0,1,1,0,498235547685756928,NaN,en
5,2014-08-10 03:42:21+00:00,498313376888799232,@HighPlainsCoot fuck. Off.,"[0, 26]",{},NaN,"<a href=""http://twitter.com/download/iphone"" r...",498312902928244736,2.437571e+09,HighPlainsRover,1539100118,0,0,2,0,498235547685756928,NaN,en
6,2014-08-11 10:58:25+00:00,498785502062190592,@AlexisAStevens yeah that tweet was written be...,"[0, 101]",{},NaN,"<a href=""http://twitter.com/download/iphone"" r...",498710112358244352,4.815092e+08,AlexisAurora_,1539100118,0,0,0,0,498235547685756928,NaN,en
7,2014-08-11 15:15:42+00:00,498850250048946176,@annaxsweat after several hours someone noticed,"[0, 47]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",498235547685756928,1.539100e+09,annaxsweat,2556027474,0,0,0,0,498235547685756928,NaN,en
8,2014-08-09 22:33:06+00:00,498235547685756928,Black teenage boys are not men. They are child...,"[0, 128]","{'hashtags': [{'text': 'ferguson', 'indices': ...",NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",nan,NaN,NaN,1539100118,177,100,9,0,498235547685756928,NaN,en
9,2014-08-09 23:04:01+00:00,498243332204949504,@annaxsweat @NeoSoulPol Same thing when #Trayv...,"[0, 131]","{'hashtags': [{'text': 'Trayvon', 'indices': [...",NaN,"<a href=""https://about.twitter.com/products/tw...",498235547685756928,1.539100e+09,annaxsweat,587926686,1,0,0,0,498235547685756928,NaN,en


In [10]:
twitter_conversations = []
errors = 0
for conv_id in tqdm(twitter_conversation_ids):
    conversation_path = os.path.join(twitter_conversations_path, conv_id, "tweets.jsonl")
    try:
        current= pd.read_json(conversation_path,
                         lines= True,
                         dtype={'in_reply_to_status_id_str':str, 'conversation_id_str': str, 'full_text':str, 'id_str':str})
        twitter_conversations.append(current)
    except:
        errors+=1
        pass

s = f"total number of loaded raw conversations:{len(twitter_conversations)}" \
f"\nnumber of conversations that failed to load: {errors}"
print(s)


total number of loaded raw conversations:297
number of conversations that failed to load: 0


## Transform to Conversation object and to feature set

In [11]:
twitter_reader = TwitterConversationReader(merge_sequential=True)

In [12]:
index = 0
conv_df = twitter_conversations[index]
conv_df["in_reply_to_status_id_str"].loc[conv_df["in_reply_to_status_id_str"] == "nan"] = None
conv = twitter_reader.parse(conv_df, root_id=twitter_conversation_ids[index])
conv

c:\users\ronp\py37-dasci\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


24165761 - 498280126254428160
├── 251275638 - 498323335965868032
└── 981050484 - 498282494387843072

In [13]:
conversations = []
bad = 0
for i, conv in enumerate(tqdm(twitter_conversations)):
    if "in_reply_to_status_id_str" not in conv:
        bad += 1
        continue

    conv_id = twitter_conversation_ids[i]
    conv["in_reply_to_status_id_str"].loc[conv["in_reply_to_status_id_str"] == "nan"] = None
    conversation = twitter_reader.parse(conv, root_id=conv_id)
    conversations.append(conversation)

s = f"total number of conversations:{len(conversations)}" \
f"\nnumber of bad conversations: {bad}"
print(s)


total number of conversations:258
number of bad conversations: 39


In [14]:
convs = pd.concat(conversation_to_dataframe(conv) for conv in conversations)
convs

,author,parent_id,depth,is_root,is_absolute_root,is_leaf,timestamp,conversation_id,full_conv_id,data.full_text,data.conversation_id_str,data.in_reply_to_screen_name
node_id,,,,,,,,,,,,
498280126254428160,24165761,None,0,True,True,False,1.407634e+09,498280126254428160,498280126254428160,[Mike Brown was staying with his grandmother f...,[498280126254428160],[nan]
498323335965868032,251275638,498280126254428160,1,False,False,True,1.407645e+09,498280126254428160,498280126254428160,[“@MichaelSkolnik: Mike Brown was staying with...,[498280126254428160],[MichaelSkolnik]
498282494387843072,981050484,498280126254428160,1,False,False,True,1.407635e+09,498280126254428160,498280126254428160,[@MichaelSkolnik Unbelieveable ..... We R in d...,[498280126254428160],[MichaelSkolnik]
498430783699554305,24165761,None,0,True,True,False,1.407670e+09,498430783699554305,498430783699554305,[Witness: Police allegedly stopped Mike Brown ...,[498430783699554305],[nan]
498440508256292864,61934810,498430783699554305,1,False,False,False,1.407672e+09,498430783699554305,498430783699554305,[@MichaelSkolnik @Agent_Kindi @Supreme___Power...,[498430783699554305],[MichaelSkolnik]
...,...,...,...,...,...,...,...,...,...,...,...,...
581473907275915264,1895019528,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit terrorism],[581473088249958400],[jjauthor]
581474582302949376,2983617889,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit Hmmm.Doesn't pundit ...,[581473088249958400],[jjauthor]
581473799398359040,2920578401,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit. I knew it! Those cr...,[581473088249958400],[jjauthor]


## Running MAXCUT

In [15]:
def expand_classification_to_neighbors(
        interactions_graph: nx.Graph,
        start_label: bool,
        start_node: str,
        all_predicted: Tuple[List[str], List[str]],
        predicted_nodes: Set[str]
):
    for node in interactions_graph.neighbors(start_node):
            if node in predicted_nodes:
                continue

            prev_node = start_node
            label = int(start_label)
            all_predicted[label].append(node)
            neighbors = list(interactions_graph.neighbors(node))
            while len(neighbors) > 1:
                next_node = neighbors[0] if neighbors[0] != prev_node else neighbors[1]
                prev_node = node
                node = next_node
                label = 1 - label
                all_predicted[label].append(node)
                neighbors = list(interactions_graph.neighbors(node))

def expand_classification(
        interactions_graph: nx.Graph,
        supporters: Sequence[str],
        opposers: Sequence[str]
) -> Tuple[List[str], List[str]]:
    predicted_nodes = set(supporters) | set(opposers)
    all_predicted = (list(supporters), list(opposers))
    for i, group in enumerate([supporters, opposers]):
        for node in group:
            expand_classification_to_neighbors(
                interactions_graph,
                1 - i,
                node,
                all_predicted,
                predicted_nodes
            )

    return all_predicted

In [44]:
rumers_author_stance = []
for conversation in tqdm(conversations):
    reply_interactions_parser = get_reply_interactions_parser()
    interaction_graph = reply_interactions_parser.parse(conversation)
    orig_graph = interaction_graph.graph.copy()
    interaction_graph = interaction_graph.get_core_interactions(inplace=True)
    if interaction_graph.graph.order() == 0:
        supporters = [conversation.root.author]
        opposers = []
    else:
        interaction_graph.set_interaction_weights(lambda x: x['replies'])
        maxcut = MaxcutStanceClassifier()
        maxcut.set_input(interaction_graph.graph)
        op = conversation.root.author
        maxcut.classify_stance(op)
        supporters = maxcut.supporters
        opposers = maxcut.complement
        
    # supporters, opposers = expand_classification(orig_graph,
    #                                              supporters,
    #                                              opposers)

    for supporter in supporters:
        row = {'conversation_id': conversation.id,
               'op': conversation.op,
               'author': supporter,
               'stance': 'supporter'}
        rumers_author_stance.append(row)
    for opposer in opposers:
        row = {'conversation_id': conversation.id,
               'op': conversation.op,
               'author': opposer,
               'stance': 'opposer'}
        rumers_author_stance.append(row)

len(rumers_author_stance)

330

In [45]:
user_stance_rumors = pd.DataFrame(rumers_author_stance)
user_stance_rumors = user_stance_rumors.rename({'conversation_id': 'threadid', 'stance':'stance_prediction'}, axis=1)
user_stance_rumors

,threadid,op,author,stance_prediction
0,498280126254428160,24165761,24165761,supporter
1,498430783699554305,24165761,571796080,supporter
2,498430783699554305,24165761,24165761,supporter
3,498430783699554305,24165761,402355096,supporter
4,498430783699554305,24165761,61934810,opposer
...,...,...,...,...
325,581047170637381632,15037033,15037033,supporter
326,581063377226637312,5943072,5943072,supporter
327,581293286268129280,288283415,288283415,supporter
328,581386094337474560,15754281,15754281,supporter


## Gold Standard

In [46]:
rumors_gs['responsetype-vs-source'].value_counts()


comment                        2923
agreed                          645
appeal-for-more-information     361
disagreed                       334
Name: responsetype-vs-source, dtype: int64

In [47]:
rumors_gold_label = rumors_gs.filter(['threadid', 'tweetid', 'responsetype-vs-source'], axis=1).rename({'responsetype-vs-source': 'label'}, axis=1)
rumors_gold_label.threadid = rumors_gold_label.threadid.astype('str')
rumors_gold_label.tweetid = rumors_gold_label.tweetid.astype('str')
rumors_gold_label

,threadid,tweetid,label
0,577258317942149120,577260238316830721,comment
1,577258317942149120,577282462788677634,comment
2,577258317942149120,577283431089901568,comment
3,577258317942149120,577284294051123201,comment
4,577258317942149120,577287160329465856,comment
...,...,...,...
4258,521346721226711040,521371539082858496,comment
4259,521346721226711040,521371742942797825,comment
4260,521346721226711040,521353434881327104,comment
4261,521346721226711040,521366121484918785,disagreed


In [48]:
twitter_convs = convs.copy()
twitter_convs = twitter_convs.reset_index().rename({
    "full_conv_id": "threadid",
    "node_id": "tweetid"
}, axis=1)
twitter_convs

,tweetid,author,parent_id,depth,is_root,is_absolute_root,is_leaf,timestamp,conversation_id,threadid,data.full_text,data.conversation_id_str,data.in_reply_to_screen_name
0,498280126254428160,24165761,None,0,True,True,False,1.407634e+09,498280126254428160,498280126254428160,[Mike Brown was staying with his grandmother f...,[498280126254428160],[nan]
1,498323335965868032,251275638,498280126254428160,1,False,False,True,1.407645e+09,498280126254428160,498280126254428160,[“@MichaelSkolnik: Mike Brown was staying with...,[498280126254428160],[MichaelSkolnik]
2,498282494387843072,981050484,498280126254428160,1,False,False,True,1.407635e+09,498280126254428160,498280126254428160,[@MichaelSkolnik Unbelieveable ..... We R in d...,[498280126254428160],[MichaelSkolnik]
3,498430783699554305,24165761,None,0,True,True,False,1.407670e+09,498430783699554305,498430783699554305,[Witness: Police allegedly stopped Mike Brown ...,[498430783699554305],[nan]
4,498440508256292864,61934810,498430783699554305,1,False,False,False,1.407672e+09,498430783699554305,498430783699554305,[@MichaelSkolnik @Agent_Kindi @Supreme___Power...,[498430783699554305],[MichaelSkolnik]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,581473907275915264,1895019528,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit terrorism],[581473088249958400],[jjauthor]
4263,581474582302949376,2983617889,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit Hmmm.Doesn't pundit ...,[581473088249958400],[jjauthor]
4264,581473799398359040,2920578401,581473088249958400,1,False,False,True,1.427469e+09,581473088249958400,581473088249958400,[@jjauthor @gatewaypundit. I knew it! Those cr...,[581473088249958400],[jjauthor]
4265,581482916645076993,373569258,581473088249958400,1,False,False,True,1.427471e+09,581473088249958400,581473088249958400,[@jjauthor @DesireeAaron @gatewaypundit this i...,[581473088249958400],[jjauthor]


In [49]:
convs_a = twitter_convs.filter(['threadid', 'tweetid', 'author'], axis=1)
rumors_gold_label_ = rumors_gold_label.merge(convs_a, how = 'left', on=['threadid', 'tweetid'])
rumors_gold_label_nona = rumors_gold_label_.dropna()
rumors_gold_label_nona

,threadid,tweetid,label,author
5,576755174531862529,576755963597746176,comment,376884268
7,576755174531862529,576758668026245120,appeal-for-more-information,5469742
8,576319832800555008,576321273594294272,comment,1946971338
10,576319832800555008,576321856166346752,disagreed,2406844921
12,576319832800555008,576327925869506560,appeal-for-more-information,138681055
...,...,...,...,...
4219,524932935137628160,524940370321629184,comment,37632713
4253,521346721226711040,521367394108071936,agreed,2747372070
4254,521346721226711040,521367905062379521,disagreed,430026970
4255,521346721226711040,521368022880378882,comment,2747372070


In [50]:
predictions = user_stance_rumors.merge(rumors_gold_label_nona, how='left', on = ['threadid', 'author'])
predictions_ = predictions.dropna()
predictions

,threadid,op,author,stance_prediction,tweetid,label
0,498280126254428160,24165761,24165761,supporter,NaN,NaN
1,498430783699554305,24165761,571796080,supporter,498504798971650049,comment
2,498430783699554305,24165761,24165761,supporter,NaN,NaN
3,498430783699554305,24165761,402355096,supporter,498451864112074752,comment
4,498430783699554305,24165761,61934810,opposer,498440508256292864,comment
...,...,...,...,...,...,...
432,581047170637381632,15037033,15037033,supporter,NaN,NaN
433,581063377226637312,5943072,5943072,supporter,NaN,NaN
434,581293286268129280,288283415,288283415,supporter,581298086246395904,comment
435,581386094337474560,15754281,15754281,supporter,NaN,NaN


## priors

In [51]:
relevant_preds = predictions_[(predictions_["label"] == "agreed") | (predictions_["label"] == "disagreed")]
relevant_preds

,threadid,op,author,stance_prediction,tweetid,label
26,500286058664579072,972651,14227485,supporter,500289420730597377,disagreed
32,500286058664579072,972651,2423249816,opposer,500290256928014336,agreed
37,500290456845299714,1363520268,796814947,opposer,500297973905895424,agreed
42,500308076004929537,1363520268,1363520268,supporter,500318691909959681,disagreed
53,500332933098385408,360738490,360738490,supporter,500345018926645250,disagreed
62,500377906305327104,23831448,23831448,supporter,500379473326665728,agreed
119,524995771587108864,88944711,88944711,supporter,525047302663442433,agreed
132,529540733020405760,842564341,842564341,supporter,529550101925343233,agreed
156,544277860555710464,872683897,872683897,supporter,544365478224101377,disagreed
163,544287209730236416,1855737487,20952653,opposer,544292469866000387,agreed


In [52]:
from collections import Counter
c = Counter(relevant_preds["label"])
c

Counter({'disagreed': 16, 'agreed': 15})

In [53]:
total_labels = len(relevant_preds)
agree_percentage = c["agreed"] / total_labels
disagree_percentage = c["disagreed"] / total_labels
agree_percentage, disagree_percentage

(0.4838709677419355, 0.5161290322580645)

In [54]:
c = Counter(relevant_preds["stance_prediction"])
c

Counter({'supporter': 16, 'opposer': 15})

In [55]:
support_percentage = c["supporter"] / total_labels
oppose_percentage = c["opposer"] / total_labels
support_percentage, oppose_percentage

(0.5161290322580645, 0.4838709677419355)

### Calculate expected predictions

In [56]:
exp_tp = agree_percentage * support_percentage
exp_tn = disagree_percentage * oppose_percentage
exp_fp = disagree_percentage * support_percentage
exp_fn = agree_percentage * oppose_percentage

print(f"exp_tp: {int(exp_tp * total_labels)}\t{exp_tp}")
print(f"exp_tn: {int(exp_tn * total_labels)}\t{exp_tn}")
print(f"exp_fp: {int(exp_fp * total_labels)}\t{exp_fp}")
print(f"exp_fn: {int(exp_fn * total_labels)}\t{exp_fn}")

exp_tp: 7	0.2497398543184183
exp_tn: 7	0.2497398543184183
exp_fp: 8	0.26638917793964617
exp_fn: 7	0.2341311134235172


In [57]:
tp = predictions_.loc[(predictions_.stance_prediction == 'supporter') & (predictions_.label == 'agreed') ]
tn = predictions_.loc[(predictions_.stance_prediction == 'opposer') & (predictions_.label == 'disagreed') ]
fp = predictions_.loc[(predictions_.stance_prediction == 'supporter') & (predictions_.label == 'disagreed') ]
fn = predictions_.loc[(predictions_.stance_prediction == 'opposer') & (predictions_.label == 'agreed') ]

print(f"tp: {len(tp)}\t{len(tp) / total_labels}")
print(f"tn: {len(tn)}\t{len(tn) / total_labels}")
print(f"fp: {len(fp)}\t{len(fp) / total_labels}")
print(f"fn: {len(fn)}\t{len(fn) / total_labels}")


tp: 10	0.3225806451612903
tn: 10	0.3225806451612903
fp: 6	0.1935483870967742
fn: 5	0.16129032258064516


## Extra for TSNE + PCA

In [ ]:
# data = pd.DataFrame.from_records(features, index="conv_id")
# print(data.head())

In [ ]:
data_ = data.loc[data.num_nodes > 10]

In [ ]:
data_.count()

In [ ]:
data_

In [ ]:
data_.describe()

In [ ]:
data_ = data_.fillna(0)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


In [ ]:
standardized_data = StandardScaler().fit_transform(data_)
print(standardized_data.shape)

In [ ]:
model = TSNE(n_components=2, random_state=0)

In [ ]:
tsne_data = model.fit_transform(standardized_data)

In [ ]:
tsne_df = pd.DataFrame(data=tsne_data, columns=("Dim_1", "Dim_2"))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.FacetGrid(tsne_df, height=9).map(plt.scatter, 'Dim_1', 'Dim_2').add_legend()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN


In [ ]:
m = DBSCAN()


In [ ]:
m.fit(standardized_data)

In [ ]:
clusters = m.labels_

In [ ]:
pd.Series(clusters).value_counts()

In [ ]:
# pca

from sklearn.decomposition import PCA


In [ ]:
pca = PCA(n_components=2)


In [ ]:
principalComponents = pca.fit_transform(standardized_data)

In [ ]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
sns.FacetGrid(principalDf, height=6).map(plt.scatter, 'principal component 1', 'principal component 2').add_legend()
plt.show()